SINAV ÇALIŞMASIDIR

In [ ]:
# KOD BLOĞU #1: Kütüphaneleri İçe Aktarma ve Veriyi Yükleme

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Makine öğrenmesi için gerekli araçlar
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, recall_score, precision_score

# Kullanacağımız 3 farklı tahmin modeli
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# CSV dosyasından müşteri verilerini okuyoruz
df = pd.read_csv("data/Telco-Customer-Churn.csv")

# KOLON NUMARALAMA SİSTEMİ
def create_column_mapping(dataframe):
    """
    DataFrame'deki tüm kolonları numaralandırır.
    Returns: {kolon_no: kolon_adı} sözlüğü
    """
    return {i+1: col for i, col in enumerate(dataframe.columns)}

def get_column_name(col_map, col_no):
    """Kolon numarasından kolon adını döndürür."""
    return col_map.get(col_no, None)

def get_column_number(col_map, col_name):
    """Kolon adından kolon numarasını döndürür."""
    for num, name in col_map.items():
        if name == col_name:
            return num
    return None

# İlk kolon haritasını oluştur
COLUMN_MAP = create_column_mapping(df)

# Kolon listesini yazdır
print("=" * 60)
print("KOLON NUMARALARI")
print("=" * 60)
for num, name in COLUMN_MAP.items():
    print(f"  [{num:2}] {name}")
print("=" * 60)

# İlk 5 satırı inceliyoruz
df.head()

In [ ]:
# KOD BLOĞU #2: Veriyi İnceleme ve Keşfetme (KOLON NUMARALARIYLA)

print("=" * 80)
print("VERİ SETİ ANALİZİ")
print("=" * 80)

# Veri boyutu bilgisi
print(f"\nVeri Boyutu: {df.shape[0]} satır, {df.shape[1]} kolon")

# Veri tiplerini inceleme
print("\n" + "-" * 80)
print("KOLON TİP ANALİZİ")
print("-" * 80)

def infer_real_dtype(series):
    """
    Bir kolonun gerçek veri tipini tahmin eder.
    İlk 5 ve son 5 satırdaki değerlere bakarak karar verir.
    """
    sample = pd.concat([series.head(5), series.tail(5)]).dropna()

    if len(sample) == 0:
        return 'unknown'

    # Sayıya çevirmeyi dene
    try:
        numeric_sample = pd.to_numeric(sample.astype(str).str.strip(), errors='coerce')
        success_rate = numeric_sample.notna().sum() / len(sample)

        if success_rate >= 0.8:
            if (numeric_sample.dropna() % 1 == 0).all():
                return 'numeric_int'
            else:
                return 'numeric_float'
    except:
        pass

    # Kategorik mi kontrol et
    unique_count = series.nunique()
    total_count = len(series)

    if unique_count <= max(10, int(total_count * 0.05)):
        return 'categorical'

    # ID kontrolü (tüm değerler benzersiz mi?)
    if unique_count == total_count:
        return 'id'

    return 'text'

# Analiz sonuçlarını sakla
analysis_results = {}
id_columns = []
suspicious_columns = []
target_column = None

print(f"\n{'NO':>3} | {'KOLON ADI':<20} | {'MEVCUT TİP':<10} | {'TAHMİN':<15} | {'DURUM':<10}")

for col_no, col_name in COLUMN_MAP.items():
    current_dtype = str(df[col_name].dtype)
    inferred_type = infer_real_dtype(df[col_name])

    # Durumu belirle
    status = "OK"

    # ID sütunu mu?
    if inferred_type == 'id':
        status = "ID (Silinecek)"
        id_columns.append(col_name)

    # Tip uyumsuzluğu var mı?
    elif current_dtype == 'object' and inferred_type in ['numeric_int', 'numeric_float']:
        status = "UYUMSUZ"
        suspicious_columns.append(col_name)

    # Hedef sütun mu? (churn, target vb.)
    target_keywords = ['churn', 'target', 'label', 'class', 'outcome']
    if any(kw in col_name.lower() for kw in target_keywords):
        if df[col_name].nunique() == 2:
            status = "HEDEF"
            target_column = col_name

    # Sonuçları kaydet
    analysis_results[col_no] = {
        'name': col_name,
        'current_dtype': current_dtype,
        'inferred_type': inferred_type,
        'status': status
    }

    print(f"[{col_no:2}] | {col_name:<20} | {current_dtype:<10} | {inferred_type:<15} | {status}")

# Özet
print("\n" + "=" * 80)
print("ANALİZ ÖZETİ")

if id_columns:
    id_nums = [get_column_number(COLUMN_MAP, c) for c in id_columns]
    print(f"ID Kolonları (silinecek): {id_nums} → {id_columns}")

if suspicious_columns:
    susp_nums = [get_column_number(COLUMN_MAP, c) for c in suspicious_columns]
    print(f"Uyumsuz Kolonlar (düzeltilecek): {susp_nums} → {suspicious_columns}")

if target_column:
    target_num = get_column_number(COLUMN_MAP, target_column)
    print(f"Hedef Kolon: [{target_num}] → {target_column}")

# Uyumsuz kolonların detaylı incelenmesi
if suspicious_columns:
    print("\n" + "-" * 80)
    print("UYUMSUZ KOLON DETAYLARI")
    print("-" * 80)

    for col_name in suspicious_columns:
        col_no = get_column_number(COLUMN_MAP, col_name)
        print(f"\n[{col_no}] {col_name}:")
        print(f"    İlk 5 değer: {df[col_name].head(5).tolist()}")

        # Boşluk karakteri kontrolü
        blank_count = (df[col_name].astype(str).str.strip() == '').sum()
        if blank_count > 0:
            print(f"    Boş değer sayısı: {blank_count}")

# Hedef değişken grafiği
if target_column:
    print("\n" + "-" * 80)
    print(f"HEDEF DEĞİŞKEN DAĞILIMI: [{get_column_number(COLUMN_MAP, target_column)}] {target_column}")
    print("-" * 80)

    plt.figure(figsize=(6,4))
    sns.countplot(x=target_column, data=df)
    plt.title(f"[{get_column_number(COLUMN_MAP, target_column)}] {target_column} Dağılımı")
    plt.show()

    print(df[target_column].value_counts())

# Korelasyon Analizi
print("\\n" + "-" * 80)
print("KORELASYON ANALİZİ")
print("-" * 80)

# Sadece sayısal kolonları seç
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

if len(numeric_cols) >= 2:
    corr_matrix = df[numeric_cols].corr()

    # Heatmap çiz
    plt.figure(figsize=(12, 8))
    sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="RdBu_r",
                center=0, vmin=-1, vmax=1, cbar_kws={'label': 'Korelasyon'})
    plt.title("Sayısal Değişkenler Arası Korelasyon", fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

    # Hedef değişkenle en yüksek korelasyonları bul
    if target_column and target_column in corr_matrix.columns:
        target_corr = corr_matrix[target_column].abs().sort_values(ascending=False)
        print(f"\\n[{target_column}] ile en yüksek korelasyonlar:")
        for i, (col, val) in enumerate(target_corr.head(6).items()):
            if col != target_column:
                col_no = get_column_number(COLUMN_MAP, col)
                print(f"    [{col_no}] {col}: {val:.3f}")
else:
    print("ℹ️  Yeteri kadar sayısal kolon bulunamadı.")

print("\\n" + "=" * 80)


In [ ]:
# KOD BLOĞU #3: Veri Temizleme ve Hazırlama (KOLON NUMARALARIYLA)

print("=" * 80)
print("VERİ TEMİZLEME")

# 1. ID KOLONLARINI SİL
print("\n[1] ID Kolonlarının Silinmesi:")
print("-" * 40)

if id_columns:
    for col_name in id_columns:
        col_no = get_column_number(COLUMN_MAP, col_name)
        print(f"    [{col_no}] {col_name} siliniyor...")
    df = df.drop(id_columns, axis=1)
    print(f"    Silinen kolon sayısı: {len(id_columns)}")
else:
    print("    ID kolonu bulunamadı")

# Kolon haritasını güncelle
COLUMN_MAP = create_column_mapping(df)

# 2. UYUMSUZ KOLONLARI DÜZELT
print("\n[2] Uyumsuz Kolonların Düzeltilmesi:")
print("-" * 40)

if suspicious_columns:
    for col_name in suspicious_columns:
        if col_name in df.columns:
            col_no = get_column_number(COLUMN_MAP, col_name)

            # Sayıya çevir
            df[col_name] = pd.to_numeric(df[col_name], errors='coerce')

            # Eksik değerleri medyan ile doldur
            missing_count = df[col_name].isna().sum()
            if missing_count > 0:
                median_val = df[col_name].median()
                df[col_name] = df[col_name].fillna(median_val)
                print(f"    [{col_no}] {col_name}: {missing_count} eksik değer medyan ({median_val:.2f}) ile dolduruldu")
            else:
                print(f"    [{col_no}] {col_name}: Sayıya çevrildi, eksik değer yok")
else:
    print("    Düzeltilecek kolon bulunamadı")

# 3. HEDEF DEĞİŞKENİ DÖNÜŞTÜR
print("\n[3] Hedef Değişken Dönüşümü:")
print("-" * 40)

if target_column and target_column in df.columns:
    target_no = get_column_number(COLUMN_MAP, target_column)
    print(f"    Hedef kolon: [{target_no}] {target_column}")
    print(f"    Mevcut değerler: {df[target_column].unique().tolist()}")

    # Eğer object tipindeyse sayıya çevir
    if df[target_column].dtype == 'object':
        unique_vals = df[target_column].unique()
        positive_keywords = ['yes', 'true', '1', 'positive', 'churn']

        target_map = {}
        for val in unique_vals:
            if str(val).lower() in positive_keywords:
                target_map[val] = 1
            else:
                target_map[val] = 0

        df[target_column] = df[target_column].map(target_map)
        print(f"    Dönüşüm: {target_map}")
    else:
        print(f"    Zaten sayısal tipte: {df[target_column].dtype}")
else:
    # Hedef kolon bulunamadıysa Churn'ü ara
    if 'Churn' in df.columns:
        target_column = 'Churn'
        target_no = get_column_number(COLUMN_MAP, target_column)
        print(f"    Varsayılan hedef: [{target_no}] {target_column}")

        if df[target_column].dtype == 'object':
            target_map = {'Yes': 1, 'No': 0}
            df[target_column] = df[target_column].map(target_map)
            print(f"    Dönüşüm: {target_map}")

# 4. KATEGORİK VERİLERİ DÖNÜŞTÜR
print("\n[4] Kategorik Verilerin Dönüştürülmesi (One-Hot Encoding):")
print("-" * 40)

# Object tipindeki kolonları bul (hedef hariç)
categorical_cols = df.select_dtypes(include='object').columns.tolist()

if target_column in categorical_cols:
    categorical_cols.remove(target_column)

if categorical_cols:
    cat_nums = [get_column_number(COLUMN_MAP, c) for c in categorical_cols]
    print(f"    Kategorik kolonlar: {cat_nums}")
    print(f"    Kolon adları: {categorical_cols}")

    # One-Hot Encoding uygula
    df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)
    print(f"    One-Hot Encoding uygulandı")
    print(f"    Yeni kolon sayısı: {len(df.columns)}")
else:
    print("    Kategorik kolon bulunamadı")

# Kolon haritasını güncelle
COLUMN_MAP = create_column_mapping(df)

# 5. ÖZELLİK MÜHENDİSLİĞİ (Feature Engineering)
print("\n[5] Özellik Mühendisliği:")
print("-" * 40)

new_features_created = False

# Tenure varsa yıl grupları oluştur
if 'tenure' in df.columns:
    print("    • Tenure Yıl Grupları oluşturuluyor...")

    def tenure_to_year_group(months):
        if months <= 12: return "0-1_Yil"
        elif months <= 24: return "1-2_Yil"
        elif months <= 36: return "2-3_Yil"
        elif months <= 48: return "3-4_Yil"
        else: return "4+_Yil"

    df['NEW_TenureYear'] = df['tenure'].apply(tenure_to_year_group)
    print(f"      Oluşturuldu: NEW_TenureYear")
    print(f"      Dağılım: {dict(df['NEW_TenureYear'].value_counts())}")
    new_features_created = True

# Servis sayısı hesapla
service_cols = [col for col in df.columns if any(s in col.lower()
                for s in ['phone', 'internet', 'security', 'backup',
                         'protection', 'support', 'streaming'])]

if service_cols:
    print("    • Toplam Servis Sayısı hesaplanıyor...")
    # Her kolon için 'Yes' veya 1 içerip içermediğini kontrol et
    df['NEW_TotalServices'] = df[service_cols].apply(
        lambda row: sum(1 for val in row if str(val).lower() in ['yes', '1', 'true']),
        axis=1
    )
    print(f"      Oluşturuldu: NEW_TotalServices")
    print(f"      Ortalama servis: {df['NEW_TotalServices'].mean():.2f}")
    print(f"      Min: {df['NEW_TotalServices'].min()}, Max: {df['NEW_TotalServices'].max()}")
    new_features_created = True

if new_features_created:
    # Yeni kategorik özellikleri encode et
    new_cat_cols = [col for col in df.columns if col.startswith('NEW_') and df[col].dtype == 'object']
    if new_cat_cols:
        df = pd.get_dummies(df, columns=new_cat_cols, drop_first=True)
        print(f"    • Yeni kategorik özellikler encode edildi")

    # Kolon haritasını güncelle
    COLUMN_MAP = create_column_mapping(df)
    print(f"Toplam {len([c for c in df.columns if c.startswith('NEW_')])} yeni özellik eklendi")
else:
    print("Yeni özellik oluşturulamadı (gerekli kolonlar bulunamadı)")

# Son durum
print("\n" + "=" * 80)
print("TEMİZLEME SONRASI VERİ")
print(f"Boyut: {df.shape[0]} satır, {df.shape[1]} kolon")
print(f"\nİlk 5 kolon:")
for i, (num, name) in enumerate(COLUMN_MAP.items()):
    if i >= 5:
        print(f"    ... ve {len(COLUMN_MAP) - 5} kolon daha")
        break
    print(f"    [{num}] {name}")

df.head()

In [ ]:
# KOD BLOĞU #4: Veriyi Eğitim ve Test Olarak Ayırma (KOLON NUMARALARIYLA)

print("=" * 80)
print("VERİ BÖLME VE ÖLÇEKLENDİRME")
print("=" * 80)

# 1. HEDEF KOLONU BUL
print("\n[1] Hedef Kolon:")
print("-" * 40)

# target_column değişkeni Kod Bloğu #2'den geliyor
if target_column and target_column in df.columns:
    target_no = get_column_number(COLUMN_MAP, target_column)
    print(f"    Hedef: [{target_no}] {target_column}")
else:
    # Churn'ü ara
    target_column = 'Churn'
    target_no = get_column_number(COLUMN_MAP, target_column)
    print(f"    Varsayılan hedef: [{target_no}] {target_column}")

# 2. X VE Y AYIR
print("\n[2] Özellik (X) ve Hedef (y) Ayrımı:")
print("-" * 40)

y = df[target_column]
X = df.drop(target_column, axis=1)

print(f"    Hedef (y): [{target_no}] {target_column}")
print(f"    Özellik (X): {X.shape[1]} kolon")
print(f"    Hedef dağılımı: 0 → {(y == 0).sum()}, 1 → {(y == 1).sum()}")

# 3. DENGESİZLİK KONTROLÜ
print("\n[3] Dengesizlik Kontrolü:")
print("-" * 40)

class_ratio = y.value_counts(normalize=True)
minority_ratio = class_ratio.min()

if minority_ratio < 0.3:
    print(f"    Dengesiz veri tespit edildi!")
    print(f"    Azınlık sınıf oranı: {minority_ratio:.1%}")
    print(f"    stratify=y kullanılacak")
else:
    print(f"    Veri dengeli (azınlık oranı: {minority_ratio:.1%})")

# 4. EĞİTİM/TEST BÖLME
print("\n[4] Eğitim/Test Bölme:")
print("-" * 40)

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print(f"    Eğitim seti: {len(X_train)} satır (%80)")
print(f"    Test seti: {len(X_test)} satır (%20)")

# 5. ÖLÇEKLENDİRME
print("\n[5] Özellik Ölçeklendirme (StandardScaler):")
print("-" * 40)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(f"    Tüm özellikler ölçeklendirildi")
print(f"    Ortalama: 0, Standart Sapma: 1")

print("\n" + "=" * 80)

In [ ]:
# KOD BLOĞU #5: Model 1 - Logistic Regression
# En basit sınıflandırma modeli
# Her özelliğe ağırlık verir, ağırlıklı toplam 0.5'ten büyükse → Churn=1

print("=" * 80)
print("MODEL 1: LOGISTIC REGRESSION")

# Modeli oluştur ve eğit
log_model = LogisticRegression(max_iter=1000)
log_model.fit(X_train, y_train)

# Tahmin yap
y_pred_log = log_model.predict(X_test)

# Sonuçları kaydet
log_accuracy = accuracy_score(y_test, y_pred_log)

print(f"\nAccuracy: {log_accuracy:.4f} ({log_accuracy*100:.2f}%)")
print("\nDetaylı Rapor:")
print(classification_report(y_test, y_pred_log))

# Hata Matrisi
plt.figure(figsize=(5,4))
sns.heatmap(confusion_matrix(y_test, y_pred_log), annot=True, fmt='d', cmap='Blues')
plt.title("Logistic Regression - Hata Matrisi")
plt.ylabel('Gerçek')
plt.xlabel('Tahmin')
plt.show()

In [ ]:
# KOD BLOĞU #6: Model 2 - Random Forest
# Birden fazla karar ağacı oluşturur (100 ağaç)
# Tüm ağaçlar oy kullanır, çoğunluk kazanır

print("=" * 80)
print("MODEL 2: RANDOM FOREST")

# Modeli oluştur ve eğit
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Tahmin yap
y_pred_rf = rf_model.predict(X_test)

# Sonuçları kaydet
rf_accuracy = accuracy_score(y_test, y_pred_rf)

print(f"\nAccuracy: {rf_accuracy:.4f} ({rf_accuracy*100:.2f}%)")
print("\nDetaylı Rapor:")
print(classification_report(y_test, y_pred_rf))

# Hata Matrisi
plt.figure(figsize=(5,4))
sns.heatmap(confusion_matrix(y_test, y_pred_rf), annot=True, fmt='d', cmap='Greens')
plt.title("Random Forest - Hata Matrisi")
plt.ylabel('Gerçek')
plt.xlabel('Tahmin')
plt.show()

In [ ]:
# KOD BLOĞU #7: Model 3 - XGBoost
# Gradient boosting tekniği kullanır
# Her yeni ağaç, önceki ağacın hatalarını düzeltmeye çalışır

print("=" * 80)
print("MODEL 3: XGBOOST")
print("=" * 80)

# Modeli oluştur ve eğit
# logloss: Modelin tahminlerindeki güveni de ölçer
xgb_model = XGBClassifier(eval_metric='logloss', random_state=42)
xgb_model.fit(X_train, y_train)

# Tahmin yap
y_pred_xgb = xgb_model.predict(X_test)

# Sonuçları kaydet
xgb_accuracy = accuracy_score(y_test, y_pred_xgb)

print(f"\nAccuracy: {xgb_accuracy:.4f} ({xgb_accuracy*100:.2f}%)")
print("\nDetaylı Rapor:")
print(classification_report(y_test, y_pred_xgb))

# Hata Matrisi
plt.figure(figsize=(5,4))
sns.heatmap(confusion_matrix(y_test, y_pred_xgb), annot=True, fmt='d', cmap='Oranges')
plt.title("XGBoost - Hata Matrisi")
plt.ylabel('Gerçek')
plt.xlabel('Tahmin')
plt.show()

In [ ]:
# KOD BLOĞU #8: Model Karşılaştırma ve Sonuç
print("=" * 80)
print("GELİŞMİŞ MODEL KARŞILAŞTIRMA")
print("=" * 80)

# Tüm metrikleri hesapla
results = {
    'Model': ['Logistic Regression', 'Random Forest', 'XGBoost'],
    'Accuracy': [
        log_accuracy,
        rf_accuracy,
        xgb_accuracy
    ],
    'F1 Score': [
        f1_score(y_test, y_pred_log),
        f1_score(y_test, y_pred_rf),
        f1_score(y_test, y_pred_xgb)
    ],
    'Recall': [
        recall_score(y_test, y_pred_log),
        recall_score(y_test, y_pred_rf),
        recall_score(y_test, y_pred_xgb)
    ],
    'Precision': [
        precision_score(y_test, y_pred_log),
        precision_score(y_test, y_pred_rf),
        precision_score(y_test, y_pred_xgb)
    ]
}

results_df = pd.DataFrame(results)

print("\n📊 DETAYLI PERFORMANS TABLOSU:")
print("-" * 90)
print(results_df.to_string(index=False))

# En iyi modeli F1 Score'a göre seç (daha dengeli metrik)
best_idx = results_df['F1 Score'].idxmax()
best_model_name = results_df.iloc[best_idx]['Model']
best_f1 = results_df.iloc[best_idx]['F1 Score']
best_accuracy = results_df.iloc[best_idx]['Accuracy']
best_recall = results_df.iloc[best_idx]['Recall']

print("\n" + "=" * 90)
print(f"🏆 EN İYİ MODEL: {best_model_name}")
print(f"   Accuracy:  {best_accuracy:.4f} ({best_accuracy*100:.2f}%)")
print(f"   F1 Score:  {best_f1:.4f} (Precision ve Recall dengesi)")
print(f"   Recall:    {best_recall:.4f} (Churn'leri yakalama oranı)")
print("=" * 90)

# Görsel karşılaştırma - Tüm metrikler
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Model Karşılaştırması - Tüm Metrikler', fontsize=16, fontweight='bold')

metrics = ['Accuracy', 'F1 Score', 'Recall', 'Precision']
colors_palette = ['#3498db', '#2ecc71', '#e74c3c', '#f39c12']

for idx, (ax, metric) in enumerate(zip(axes.flat, metrics)):
    sorted_df = results_df.sort_values(metric, ascending=True)
    colors = [colors_palette[idx] if m != best_model_name else '#9b59b6' for m in sorted_df['Model']]

    bars = ax.barh(sorted_df['Model'], sorted_df[metric], color=colors)
    ax.set_xlabel(metric, fontweight='bold')
    ax.set_title(f'{metric} Karşılaştırması')

    # Değerleri göster
    for bar, val in zip(bars, sorted_df[metric]):
        ax.text(bar.get_width() + 0.005, bar.get_y() + bar.get_height()/2,
                f'{val:.4f}', va='center', fontsize=9)

    # X ekseni sınırlarını ayarla
    min_val = sorted_df[metric].min()
    max_val = sorted_df[metric].max()
    ax.set_xlim(min_val - 0.05, max_val + 0.05)

plt.tight_layout()
plt.show()

print("\nSONUÇ:")
print(f"Bu veri seti için {best_model_name} modeli en iyi performansı gösterdi.")
print(f"Model, test verisinde {best_accuracy*100:.2f}% doğruluk ve {best_f1:.4f} F1 Score'a ulaştı.")
print(f"Recall: {best_recall:.4f} - Model churn olacak müşterilerin %{best_recall*100:.1f}'ini yakalayabiliyor.")

